In [ ]:
import pandas as pd
import os
import numpy as np

from src.plotting import boxplot
from src.utils import load_results
from src.mpc_dataclass import dataclass_group_by, find_top_costs

from bokeh.io import output_notebook, show
output_notebook()

In [ ]:
MPC_retuls_path = os.path.abspath(os.path.join('Results', 'MPC_multiAvg_results.pkl'))
MPC_results = load_results(MPC_retuls_path)

In [ ]:
top_MPC_costs = []

for strategy, grouped_results in dataclass_group_by(MPC_results, by=lambda x: f'{x.P.N_DS}NDS_{x.P.DS_opts_name[12:]}'): 
    top_cost_results = find_top_costs(list(grouped_results), use_top_nns=10)
    print([results.Cost for results in top_cost_results])

    for results in top_cost_results:
        top_MPC_costs.append({
            'setup': strategy,
            'version': results.P.V_NN,
            'cost': results.Cost,
            'soltime': np.mean(results.Time) * 1e3 # convert to ms
        })

df = pd.DataFrame.from_dict(top_MPC_costs).set_index(['setup', 'version'])

df.head(20)

In [ ]:
# extract only costs
costs = df.loc[:, 'cost'].reset_index().drop('version', axis=1)
category_name = 'setup'
value_name = 'cost'
costs.head(10)

In [ ]:
show(boxplot(
    costs, 
    category_name,
    value_name,
    show_non_outliers=True, 
    show_outliers=False, 
    color_palette='Viridis10', 
    title='Costs with {} samples each'.format(costs.shape[0])
))

In [ ]:
# extract only soltimes
soltimes = df.loc[:, 'soltime'].reset_index().drop('version', axis=1)
category_name = 'setup'
value_name = 'soltime'
soltimes.head(10)

In [ ]:
show(boxplot(
    soltimes, 
    category_name,
    value_name,
    show_non_outliers=True, 
    show_outliers=True, 
    color_palette='Viridis10', 
    unit='ms', 
    title='Solving times with {} samples each'.format(costs.shape[0])
))